# Preprocess the MemeTracker Dataset

The following notebook contains the documented code used to preprocess the MemeTracker dataset for our experiments. 

**Note:** Because the dataset is quite large, a machine with at least ~30GB of RAM is necessary to run this notebook.

---

Import libs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import gzip
import pickle
import pandas as pd
import numpy as np
import multiprocessing

from datetime import datetime
import pytz

from matplotlib import pyplot as plt
%matplotlib inline

import networkx as nx

from tsvar.preprocessing import Dataset

# Set larger cell width for nicer visualization
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

---

# 0. Download and parse the dataset

First, all the files from the dataset must be downloaded from the [SNAP](http://snap.stanford.edu/data/memetracker9.html) dataset repository.

Then the raw files must be parsed using the `raw2df.py` script provided by [NPHC](https://github.com/achab/nphc/tree/master/nphc/datasets/memetracker) to format the raw data in a convenient tabular format.

The point processes can then be built using the following notebook.

---

## 1. Load the raw MemeTracker dataframe

Set the input directory where the data is located

In [3]:
DATA_DIR = '/root/workspace/var-wold/data/memetracker'

Load raw dataframes (in parallel)

In [22]:
list_df_files = sorted(glob.glob(os.path.join(DATA_DIR, 'parsed', 'df_*.csv')))

def worker(fname):
    return pd.read_csv(fname)

pool = multiprocessing.Pool(len(list_df_files))

jobs = list()
for fname in list_df_files:
    job = pool.apply_async(worker, (fname, ))
    jobs.append(job)

data = list()
for job in jobs:
    data.append(job.get())

df = pd.concat(data, ignore_index=True)
del data

pool.close()
pool.terminate()

* `Blog` = receiver
* `Hyperlink` = sender

Vizualize the dataset

In [23]:
print(df.shape)
df

(141578503, 5)


,Date,Hyperlink,Blog,PostNb,WeightOfLink
0,2008-08-01 00:00:00,,http://codeproject.com,0,1.0
1,2008-08-01 00:00:01,,http://wallstreetexaminer.com,1,1.0
2,2008-08-01 00:00:01,,http://news.bbc.co.uk,2,1.0
3,2008-08-01 00:00:01,,http://news.bbc.co.uk,3,1.0
4,2008-08-01 00:00:01,,http://news.bbc.co.uk,4,1.0
...,...,...,...,...,...
141578498,2009-05-01 00:00:00,http://foulweather.blogspot.com,http://foulweather.blogspot.com,15312736,0.2
141578499,2009-05-01 00:00:00,http://2.bp.blogspot.com,http://foulweather.blogspot.com,15312736,0.2
141578500,2009-05-01 00:00:00,http://dischord.com,http://foulweather.blogspot.com,15312736,0.2
141578501,2009-05-01 00:00:00,http://stream.publicbroadcasting.net,http://foulweather.blogspot.com,15312736,0.2


---

## 2. Clean the dataframe

### 2.1. Clean columns

#### Clean the `Hyperlink column`

In [24]:
df['Hyperlink'] = df['Hyperlink'].str.strip()  # Remove whitespaces (that appear in null hyperlinks)

#### Cast `Date` and build timestamps

In [25]:
df['Date'] = pd.to_datetime(df['Date'])
df['Timestamp'] = df['Date'].values.astype(np.int64) // (10 ** 9)

# NOTE: We don't use a time-zone here, so to get back the same datetime, we need to use: 
# df_filtered.Timestamp.apply(lambda t: datetime.fromtimestamp(t - 2*3600))

### 2.2. Filter invalid events

#### Self-references

Only keep cross-references between different sites (i.e., remove self references)

In [26]:
cross_event_mask = df.Hyperlink != df.Blog
print(f'{np.sum(cross_event_mask):,d} events are cross-sites'
      f' out of the {len(df):,d} ({np.sum(cross_event_mask)*100/len(df):.2f}%)')

111,893,425 events are cross-sites out of the 141,578,503 (79.03%)


#### Blogs with no hyperlink

In [27]:
null_hp_mask = df.Hyperlink.apply(len) == 0  # Build mask of valid hyperlinks
print(f'{np.sum(null_hp_mask):,d} posts have zero hyperlinks'
      f'out of the {len(df):,d} ({np.sum(null_hp_mask)*100/len(df):.2f}%)')

39,875,581 posts have zero hyperlinksout of the 141,578,503 (28.16%)


Filter the dataframe based on these masks

In [28]:
valid_event_mask = (
    #~null_hp_mask &     # Keep only events with non-null hyperlink
    #cross_event_mask &  # Keep only cross-site events
    np.ones(len(df)).astype(bool)
)

print(f'Keep {np.sum(valid_event_mask):,d}'
      f' out of the {len(df):,d} events ({np.sum(valid_event_mask)*100/len(df):.2f}%)')

df_filtered = df.loc[valid_event_mask]
display(df_filtered)

Keep 141,578,503 out of the 141,578,503 events (100.00%)


,Date,Hyperlink,Blog,PostNb,WeightOfLink,Timestamp
0,2008-08-01 00:00:00,,http://codeproject.com,0,1.0,1217548800
1,2008-08-01 00:00:01,,http://wallstreetexaminer.com,1,1.0,1217548801
2,2008-08-01 00:00:01,,http://news.bbc.co.uk,2,1.0,1217548801
3,2008-08-01 00:00:01,,http://news.bbc.co.uk,3,1.0,1217548801
4,2008-08-01 00:00:01,,http://news.bbc.co.uk,4,1.0,1217548801
...,...,...,...,...,...,...
141578498,2009-05-01 00:00:00,http://foulweather.blogspot.com,http://foulweather.blogspot.com,15312736,0.2,1241136000
141578499,2009-05-01 00:00:00,http://2.bp.blogspot.com,http://foulweather.blogspot.com,15312736,0.2,1241136000
141578500,2009-05-01 00:00:00,http://dischord.com,http://foulweather.blogspot.com,15312736,0.2,1241136000
141578501,2009-05-01 00:00:00,http://stream.publicbroadcasting.net,http://foulweather.blogspot.com,15312736,0.2,1241136000


Percentage of non-empty hyperlinks

In [29]:
np.sum(df_filtered.Hyperlink != '') * 100 / len(df_filtered)

71.83500308659147

Remove the raw dataframe, we only work on the filtered one from now on.

In [237]:
df_filterederederedered

,Date,Hyperlink,Blog,PostNb,WeightOfLink,Timestamp
0,2008-08-01 00:00:00,,http://codeproject.com,0,1.0,1217548800
1,2008-08-01 00:00:01,,http://wallstreetexaminer.com,1,1.0,1217548801
2,2008-08-01 00:00:01,,http://news.bbc.co.uk,2,1.0,1217548801
3,2008-08-01 00:00:01,,http://news.bbc.co.uk,3,1.0,1217548801
4,2008-08-01 00:00:01,,http://news.bbc.co.uk,4,1.0,1217548801
...,...,...,...,...,...,...
141578498,2009-05-01 00:00:00,http://foulweather.blogspot.com,http://foulweather.blogspot.com,15312736,0.2,1241136000
141578499,2009-05-01 00:00:00,http://2.bp.blogspot.com,http://foulweather.blogspot.com,15312736,0.2,1241136000
141578500,2009-05-01 00:00:00,http://dischord.com,http://foulweather.blogspot.com,15312736,0.2,1241136000
141578501,2009-05-01 00:00:00,http://stream.publicbroadcasting.net,http://foulweather.blogspot.com,15312736,0.2,1241136000


### 2.3. Find the top-100 blogs

In [30]:
df_filtered['has_hyperlink'] = df_filtered.Hyperlink != ''  # Indicate if event has hyperlink
df_filtered['has_hyperlink'] = df_filtered['has_hyperlink'].astype(int)

Build the count of number of hyperlink per blogs, i.e., how many times a blog was cited.

Build the count of number of posts per blogs.

In [31]:
count_series = df_filtered.groupby('Blog').agg({'PostNb': set})['PostNb'].apply(len)

Keep only the top-100 sites

In [32]:
top_num = 100

top_series = count_series.sort_values(ascending=False).iloc[:top_num]
print(f'There are {top_series.sum():,d} items in the top-{top_num:d} sites')
display(top_series)

There are 24,280,563 items in the top-100 sites


Blog
http://blogs.myspace.com           1721787
http://seattle.craigslist.org       908675
http://sfbay.craigslist.org         848217
http://chicago.craigslist.org       832027
http://blog.myspace.com             793208
                                    ...   
http://huffingtonpost.com            82245
http://sandmountainreporter.com      77939
http://twitter.com                   76874
http://forbes.com                    76804
http://rss.rssad.jp                  76273
Name: PostNb, Length: 100, dtype: int64

### 2.3. Keep only events between sites in the top-100 blogs

We finally remove all events coming from hyperlinks that are not part of the top-100 blogs.

In [33]:
top_site_set = set(top_series.index.tolist())  # All top blog sites

top_blog_mask = df_filtered['Blog'].isin(top_site_set)     # Blogs is in top
top_hp_mask = df_filtered['Hyperlink'].isin(set(list(top_site_set) + ['']))  # Hyperlink is in top or no hyperlink (i.e. is null)

In [241]:
# Build mask of valid events
valid_event_mask = top_blog_mask & top_hp_mask

# Filter
df_top = df_filtered.loc[valid_event_mask]
assert len(df_top) == np.sum(valid_event_mask)

print(f'{np.sum(valid_event_mask):,d} events are between the top-{top_num} sites'
      f' out of the {len(df_filtered):,d} ({np.sum(valid_event_mask)*100/len(df_filtered):.2f}%)')

15,168,774 events are between the top-100 sites out of the 141,578,503 (10.71%)


### 2.4. Final formatting steps

Build numerical index for each blog

In [242]:
top_name_to_idx_map = dict(zip(top_series.index, range(top_num)))

# Make numerical index for blogs
df_top['Blog_idx'] = df_top['Blog'].apply(lambda name: top_name_to_idx_map[name])

# Add hyperlinks index
top_name_to_idx_map[''] = None  # Set None for No-Hyperlink
df_top['Hyperlink_idx'] = df_top['Hyperlink'].apply(lambda name: top_name_to_idx_map[name]).astype(pd.Int32Dtype())

Order by time

In [11]:
df_top = df_top.sort_values(by='Timestamp')

Translate time origin

In [65]:
df_top['Timestamp'] -= df_top['Timestamp'].min()

Order columns

In [66]:
df_top = df_top[['Hyperlink_idx', 'Blog_idx', 'Hyperlink', 'Blog', 'Date', 'Timestamp']]

In [68]:
print(df_top.shape)
display(df_top.head(10))

(15168774, 6)


,Hyperlink_idx,Blog_idx,Hyperlink,Blog,Date,Timestamp
2,NaN,25,,http://news.bbc.co.uk,2008-08-01 00:00:01,0
3,NaN,25,,http://news.bbc.co.uk,2008-08-01 00:00:01,0
4,NaN,25,,http://news.bbc.co.uk,2008-08-01 00:00:01,0
320,NaN,81,,http://in.news.yahoo.com,2008-08-01 00:00:46,45
357,NaN,81,,http://in.news.yahoo.com,2008-08-01 00:00:48,47
408,NaN,45,,http://groups.google.com,2008-08-01 00:00:57,56
445,27,27,http://citeulike.org,http://citeulike.org,2008-08-01 00:01:18,77
447,27,27,http://citeulike.org,http://citeulike.org,2008-08-01 00:01:18,77
475,NaN,15,,http://us.rd.yahoo.com,2008-08-01 00:01:32,91
523,NaN,61,,http://uk.news.yahoo.com,2008-08-01 00:01:41,100


We only work with the top blogs from now on.

Save the clean dataframe

In [69]:
df_top.to_pickle(os.path.join(DATA_DIR, 'memetracker-top100-clean.pickle.gz'), compression='gzip')

In [70]:
df_top.to_csv(os.path.join(DATA_DIR, 'memetracker-top100-clean.csv.gz'), 
              sep=',', compression='gzip', header=False, index=False)

## 3. Build point process

Aggregate events per Blog

In [89]:
mmDataset = MemeTrackerDataset(os.path.join(DATA_DIR, 'memetracker-top100-clean.pickle.gz'))

t0 = mmDataset.data.Timestamp.min()
t1 = mmDataset.data.Timestamp.max()

train_start = t0 + (t1 - t0) * 0.85
train_end = t0 + (t1 - t0) * 0.90
test_end = t0 + (t1 - t0) * 0.95

train_events, train_graph, test_events, test_graph = mmDataset.build_train_test(train_start, train_end, test_end)

nodelist = sorted(train_events.keys())

train_adj = nx.adjacency_matrix(train_graph, nodelist).toarray()

In [101]:
dataset = tsvar.preprocessing.Dataset.from_pickle(os.path.join(DATA_DIR, 'dataset_memetracker_good.pk'))

In [107]:
def split_train_test(dataset, chunk_idx, chunk_total):
    # Set start and end times of train and test sets
    train_t0 = dataset.end_time * chunk_idx / chunk_total
    train_t1 = dataset.end_time * (chunk_idx + 1) / chunk_total
    test_t0 = dataset.end_time * (chunk_idx + 1) / chunk_total
    test_t1 = dataset.end_time * (chunk_idx + 2) / chunk_total
    # Filter the events in train/test observation windows
    train_events = [ev[(ev >= train_t0) & (ev < train_t1)] for ev in dataset.timestamps]
    test_events = [ev[(ev >= test_t0) & (ev < test_t1)] for ev in dataset.timestamps]
    # Remove dimensions with no nodes
    nodes_to_keep = np.array([(len(train_events[i]) > 0) & (len(test_events[i]) > 0)
                              for i in range(dataset.dim)])
    train_events = np.array(train_events)[nodes_to_keep].tolist()
    train_events = [ev - train_t0 for ev in train_events]
    test_events = np.array(test_events)[nodes_to_keep].tolist()
    test_events = [ev - test_t0 for ev in test_events]
    return train_events, test_events


In [108]:
train_events, test_events = split_train_test(dataset, chunk_idx=17, chunk_total=20)

In [109]:
sum(map(len, train_events))

822719

In [113]:
dataset.time_scale

426.3722723177017

In [112]:
min(map(min, train_events)), max(map(max, train_events))

(0.006332494340313133, 2766.030477519489)